## Screening Assignment - Ineuron

> Please do not skip any cells while executing this notebook, as the later programs depend on the previous cells, Thank you!

In [1]:
import os
import time
import functools
from shutil import move, copymode
import logging as lg
from abc import ABC, abstractmethod
import mimetypes
from tempfile import mkstemp

#### Set up logging 

In [2]:
# set up logging
logger = lg.getLogger('logger')
logger.setLevel(lg.DEBUG)

# formatter
formatter = lg.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# console handler
ch = lg.StreamHandler()
ch.setLevel(lg.WARNING)
ch.setFormatter(formatter)

# file handler
fh = lg.FileHandler('./example.log', encoding='utf-8')
fh.setFormatter(formatter)
fh.setLevel(lg.INFO)

# add handlers
logger.addHandler(fh)
logger.addHandler(ch)

### 1. Create a function in python to read the text file and replace specific content of the file.

In [3]:
def parse_text(path: str, pattern: str = 'placement', replacement: str = 'screening', log_level: str = 'INFO'):
    # init logs
    try:
        level = getattr(lg, log_level.upper())
    except AttributeError:
        logger.error('Invalid log level: %s' % log_level)
    else:
        fh.setLevel(level)

    try:

        # check if the given file exists
        logger.debug('Checking if the file path exists on system.')

        if not os.path.isfile(path):
            logger.debug('File does not exist.')
            raise FileNotFoundError('The given path is not a file.')

        logger.info('Given file exists on system.')

        # check the mimetype of the file
        logger.debug('Checking file type.')
        file_type, encoding = mimetypes.guess_type(path)

        if file_type != 'text/plain':
            logger.debug('File has invalid type.')
            raise TypeError(f"Expected {path} to be a text file with '.txt' extension")

        logger.info('File has a valid type, text/plain.')

        logger.debug('Creating temporary file.')
        fd, temp_path = mkstemp()

        logger.debug('Temporary file created, now writing to temporary file')
        with open(temp_path, 'w', encoding=encoding) as temp_file:
            with open(path, encoding=encoding) as old_file:
                for line in old_file:
                    # replace the string and write in the temp file
                    temp_file.write(line.replace(pattern, replacement))

            logger.info('Successfully wrote to the temporary file')

        # fd is file descriptor of temp fie
        os.close(fd)
        logger.debug('Temporary file closed.')

        # copy permissions of the original file to temp file
        logger.debug('Replacing original file with modified copy')
        copymode(path, temp_path)

        # remove original file
        os.remove(path)
        move(temp_path, path)
        logger.info('Successfully modified the content in given file')

    except Exception as e:
        logger.debug('An error has occured.')
        logger.exception(e)

> Note that if you have any files like example.txt , example.log in the current directory they can be modified by this program

In [4]:
## demonstrate usage

# create file

with open('./example.txt', 'w') as f:
    f.write('This is a placement assignment.')

print('Original file content: ')
with open('./example.txt') as f:
    for line in f:
        print(line)
print()
        
# modify content
parse_text('./example.txt')

print('Modified file content: ')
with open('./example.txt') as f:
    for line in f:
        print(line)

Original file content: 
This is a placement assignment.

Modified file content: 
This is a screening assignment.


### Abstract class

> Abstract base classes ABCs are used to create a blueprint for concrete subclasses, In the example I will create a validator ABC which lays down the methods to be present in a general validator class, these classes can be used to validate a data type. After this I'll create some concrete classes based upon this ABC.

In [5]:
# For these example I'll no longer log to the file, but just to the console

logger.removeHandler(fh)

In [6]:
class Validator(ABC):
    """This is an abstract class for general validator classes that may be used to validate data"""
    
    @abstractmethod
    def validate(self, value):
        ''' This is an abstract method that must be overridden in the concerete classes'''

In [7]:
# You can not initialise ABC without overriding the abstract methods, even if you create a subclass!

class SubValidator(Validator):
    
    def new_method(self):
        print('I will not get printed since the class cant be initiated.')


In [8]:
try:
    validator = SubValidator()
    
except Exception as e:
    logger.exception(e)

2022-06-08 22:13:55,632 - logger - ERROR - Can't instantiate abstract class SubValidator with abstract method validate
Traceback (most recent call last):
  File "/tmp/ipykernel_2472/3259379208.py", line 2, in <cell line: 1>
    validator = SubValidator()
TypeError: Can't instantiate abstract class SubValidator with abstract method validate


#### Create concrete class and demonstrate usage

In [9]:
# Checks if the given data is either float or int and if it is in the desired range,
# specified by max and min values

class NumberVaidator(Validator):

    def __init__(self, minvalue, maxvalue):
        
        if not isinstance(minvalue, (int, float)) or not isinstance(maxvalue, (int,float)):
            raise TypeError(f'Expected {minvalue}, {maxvalue} to be an int or float')

        self.minvalue = minvalue
        self.maxvalue = maxvalue
        

    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'Expected {value!r} to be an int or float')
        if self.minvalue is not None and value < self.minvalue:
            raise ValueError(
                f'Expected {value!r} to be at least {self.minvalue!r}'
            )
        if self.maxvalue is not None and value > self.maxvalue:
            raise ValueError(
                f'Expected {value!r} to be no more than {self.maxvalue!r}'
            )
        return True

In [10]:
# Usage

def check_price(price: float|int, min: float|int, max: float|int):
    ''' Checks if the input price is within bounds'''
    
    # use Number validator to validate inputs
    try:
        validator = NumberVaidator(minvalue=min, maxvalue=max)
        if validator.validate(price):
            print('Price is within bounds!')
    
    except ValueError:
        print('The price is out of bounds!')
        
    except TypeError as e:
        logger.exception(e)
            

In [11]:
check_price(10, min=0, max=100)

Price is within bounds!


In [12]:
check_price(1000, min=0, max=100)

The price is out of bounds!


In [13]:
check_price(10, min='a', max=1)

2022-06-08 22:14:00,893 - logger - ERROR - Expected a, 1 to be an int or float
Traceback (most recent call last):
  File "/tmp/ipykernel_2472/3951821985.py", line 8, in check_price
    validator = NumberVaidator(minvalue=min, maxvalue=max)
  File "/tmp/ipykernel_2472/119248699.py", line 9, in __init__
    raise TypeError(f'Expected {minvalue}, {maxvalue} to be an int or float')
TypeError: Expected a, 1 to be an int or float


In [14]:
check_price('xyz', min=0, max=100)

2022-06-08 22:14:01,415 - logger - ERROR - Expected 'xyz' to be an int or float
Traceback (most recent call last):
  File "/tmp/ipykernel_2472/3951821985.py", line 9, in check_price
    if validator.validate(price):
  File "/tmp/ipykernel_2472/119248699.py", line 17, in validate
    raise TypeError(f'Expected {value!r} to be an int or float')
TypeError: Expected 'xyz' to be an int or float


### Multiple Inheritance

> I'll demonstrate multiple inheritance by creating another ABC and the inheriting this ABC and the previous Validator class, at the end it can be seen that the methods from both parents will be inherited by the child class

In [15]:
class Messenger(ABC):
    
    @abstractmethod
    def show_message(self, message):
        pass


In [16]:
## create child class that inherits from multiple parents

class ChildValidator(NumberVaidator, Messenger):
    
    # init is supplied by the Validator class
    def show_message(self, message: str):
        if not isinstance(message, str):
            raise TypeError(f'Expected {message!r} to be an str')
        
        print(message)


In [17]:
## demonstrate usage

child = ChildValidator(minvalue=0, maxvalue=100)

# do number validation and show message!

def check_number(value,):
    ''' Check if number is between 0 and 100, and print result'''
    
    try:
        # use validate method from NumberValidator
        if child.validate(value):
        
            # use show_message method from Messenger class
            child.show_message('Yes the number is within 0 and 100')
    
    except ValueError:
        child.show_message('The number is not within 0 and 100')
    
    except TypeError as e:
        logger.exception(e)
    
    

In [18]:
check_number(98.6)

Yes the number is within 0 and 100


In [19]:
check_number(10000)

The number is not within 0 and 100


In [20]:
check_number('I am not number')

2022-06-08 22:14:20,825 - logger - ERROR - Expected 'I am not number' to be an int or float
Traceback (most recent call last):
  File "/tmp/ipykernel_2472/1871522971.py", line 12, in check_number
    if child.validate(value):
  File "/tmp/ipykernel_2472/119248699.py", line 17, in validate
    raise TypeError(f'Expected {value!r} to be an int or float')
TypeError: Expected 'I am not number' to be an int or float


### Decorators

> Decorators are syntactic sugar, they are used to encapsulate methods/ classes to change their behavior. I've already used @abstractmethod multiple times above. Now I'll provide a decorator example.

In [21]:
## measuring time 

def timer_decorator(func):
    ''' This decorator shows the time taken by decorated function to execute'''
    
    functools.wraps(func)
    
    def timer(*args, **kwargs):
        
        # use performance counter
        start = time.perf_counter()
        
        # execute the decorated func
        out = func(*args, **kwargs)
        
        end = time.perf_counter()
        period = end - start
        
        # show upto two digits beyong decimal
        print (f"Finished {repr(func.__name__)} in {period:.2f} seconds")
        
        # return the original value
        return out
    
    return timer

In [22]:
## demonstrate usage

def cube(n: int):
    if not isinstance(n, int) or n < 0:
        raise TypeError('Expected n to be integer not less than zero.')
    
    ## classic example of where you should not use a list but a generator
    ## this time we must use a list because we need to waste some time
    
    l = [0]
    for i in range(n):
        l.append(i**3)
    
    return sum(l)

In [23]:
cube(1000000)

249999500000250000000000

In [24]:
## let's decorate the function now and see the change

@timer_decorator
def cube(n: int):
    if not isinstance(n, int) or n < 0:
        raise TypeError('Expected n to be integer not less than zero.')
    
    ## classic example of where you should not use a list but a generator
    ## this time we must use a list because we need to waste some time
    
    l = [0]
    for i in range(n):
        l.append(i**3)
    
    return sum(l)

In [25]:
## execute decorated function

cube(1000000)

Finished 'cube' in 1.38 seconds


249999500000250000000000

> Now the function is modified to tell us about its runtime...

> The use of functools.wraps() is to make sure the function still retains its identity, without it the function will not retain its attributes like __name__ or __doc__, it will inherit those from the decoratror which is not what we want....